In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [39]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [40]:
class BrainTumorDataset(Dataset):
    def __init__(self, img_dir, mask_dir, img_size=256):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_size = img_size

        img_files = set(os.listdir(img_dir))
        mask_files = set(os.listdir(mask_dir))

        # Only keep common files
        self.files = sorted(list(img_files & mask_files))

        print(f"Total paired samples: {len(self.files)}")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]

        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)

        image = cv2.imread(img_path)
        mask  = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if image is None or mask is None:
            raise ValueError(f"Missing file: {img_name}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image = cv2.resize(image, (self.img_size, self.img_size))
        mask  = cv2.resize(mask, (self.img_size, self.img_size))

        image = image / 255.0
        mask  = mask / 255.0

        image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)
        mask  = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        return image, mask


In [41]:
class BrainTumorDataset(Dataset):
    def __init__(self, img_dir, mask_dir, img_size=256):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(img_dir))
        self.img_size = img_size

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask  = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        image = cv2.resize(image, (self.img_size, self.img_size))
        mask  = cv2.resize(mask, (self.img_size, self.img_size))

        image = image / 255.0
        mask  = mask / 255.0

        image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)
        mask  = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        return image, mask


In [42]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256), nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)


In [43]:
class CBAM(nn.Module):
    def __init__(self, ch, r=8):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(ch, ch//r),
            nn.ReLU(),
            nn.Linear(ch//r, ch)
        )
        self.spatial = nn.Conv2d(2, 1, 7, padding=3)

    def forward(self, x):
        b,c,h,w = x.size()
        avg = F.adaptive_avg_pool2d(x,1).view(b,c)
        mx  = F.adaptive_max_pool2d(x,1).view(b,c)
        ca = torch.sigmoid(self.mlp(avg)+self.mlp(mx)).view(b,c,1,1)
        x = x * ca

        avg_sp = torch.mean(x,1,keepdim=True)
        mx_sp,_ = torch.max(x,1,keepdim=True)
        sa = torch.sigmoid(self.spatial(torch.cat([avg_sp,mx_sp],1)))
        return x * sa


In [44]:
class TransformerBlock(nn.Module):
    def __init__(self, dim=256, heads=8):
        super().__init__()
        self.attn = nn.MultiheadAttention(dim, heads, batch_first=True)
        self.norm1 = nn.LayerNorm(dim)
        self.ff = nn.Sequential(
            nn.Linear(dim, dim*4),
            nn.GELU(),
            nn.Linear(dim*4, dim)
        )
        self.norm2 = nn.LayerNorm(dim)

    def forward(self, x):
        attn,_ = self.attn(x,x,x)
        x = self.norm1(x + attn)
        x = self.norm2(x + self.ff(x))
        return x


In [45]:
class GRUDecoder(nn.Module):
    def __init__(self, dim=256):
        super().__init__()
        self.gru = nn.GRU(dim, dim, batch_first=True)
        self.out = nn.Conv2d(dim, 1, 1)

    def forward(self, x, h, w):
        x,_ = self.gru(x)
        x = x.transpose(1,2).view(-1, x.size(-1), h, w)
        return torch.sigmoid(self.out(x))


In [46]:
class HybridSegNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = EncoderCNN()
        self.attn = CBAM(256)
        self.trans = TransformerBlock()
        self.decoder = GRUDecoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.attn(x)
        b,c,h,w = x.shape
        x = x.flatten(2).transpose(1,2)
        x = self.trans(x)
        return self.decoder(x,h,w)


In [47]:
def dice_bce_loss(pred, mask):
    bce = F.binary_cross_entropy(pred, mask)
    inter = (pred * mask).sum()
    dice = 1 - (2*inter + 1)/(pred.sum() + mask.sum() + 1)
    return bce + dice

def dice_score(pred, mask):
    pred = (pred > 0.5).float()
    inter = (pred * mask).sum()
    return (2*inter + 1)/(pred.sum() + mask.sum() + 1)


In [48]:
IMG_DIR = "/kaggle/input/brain-tumor-segmentation-1/brain tumor segmentation/image"
MASK_DIR = "/kaggle/input/brain-tumor-segmentation-1/brain tumor segmentation/masks"

dataset = BrainTumorDataset(IMG_DIR, MASK_DIR)
loader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=2)


In [49]:
# dataset = BrainTumorDataset(IMG_DIR, MASK_DIR)

# loader = DataLoader(
#     dataset,
#     batch_size=8,
#     shuffle=True,
#     num_workers=2,
#     pin_memory=True
# )


In [50]:
# for epoch in range(EPOCHS):
#     model.train()
#     total_loss = 0

#     for img, mask in tqdm(loader):
#         img, mask = img.to(device), mask.to(device)

#         pred = model(img)
#         loss = dice_bce_loss(pred, mask)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {total_loss/len(loader):.4f}")


In [51]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = HybridSegNet().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

EPOCHS = 25

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for img, mask in tqdm(loader):
        img, mask = img.to(device), mask.to(device)

        pred = model(img)
        loss = dice_bce_loss(pred, mask)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {total_loss/len(loader):.4f}")


  0%|          | 0/274 [00:00<?, ?it/s]
[ WARN:0@486.349] global loadsave.cpp:275 findDecoder imread_('/kaggle/input/brain-tumor-segmentation-1/brain tumor segmentation/masks/enh_608.png'): can't open/read file: check file path/integrity


error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipykernel_55/757207223.py", line 20, in __getitem__
    mask  = cv2.resize(mask, (self.img_size, self.img_size))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



[ WARN:0@486.353] global loadsave.cpp:275 findDecoder imread_('/kaggle/input/brain-tumor-segmentation-1/brain tumor segmentation/masks/enh_2382.png'): can't open/read file: check file path/integrity


In [ ]:
model.eval()
with torch.no_grad():
    img, mask = dataset[0]
    img = img.unsqueeze(0).to(device)
    pred = model(img)

    pred = pred.cpu().numpy()[0,0]


[ WARN:0@486.364] global loadsave.cpp:275 findDecoder imread_('/kaggle/input/brain-tumor-segmentation-1/brain tumor segmentation/masks/enh_2271.png'): can't open/read file: check file path/integrity
[ WARN:0@486.367] global loadsave.cpp:275 findDecoder imread_('/kaggle/input/brain-tumor-segmentation-1/brain tumor segmentation/masks/enh_426.png'): can't open/read file: check file path/integrity
